In [124]:
import sqlalchemy as sa
import sqlalchemy.ext.declarative as sad

In [125]:
engine = sa.create_engine('sqlite:///data/nobel_prize.db', echo=True)

In [126]:
Base = sad.declarative_base()

In [127]:
class Winner(Base):
    __tablename__ = 'winners'
    
    id = sa.Column(sa.Integer, primary_key=True)
    name = sa.Column(sa.String)
    category = sa.Column(sa.String)
    year = sa.Column(sa.Integer)
    nationality = sa.Column(sa.String)
    sex = sa.Column(sa.Enum('male','female'))
    
    def __repr__(self):
        return "<Winner(name='%s', category='%s', year='%s')"%(self.name, self.category, self.year)

In [128]:
Base.metadata.create_all(engine)

2016-12-20 09:03:27,498 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2016-12-20 09:03:27,499 INFO sqlalchemy.engine.base.Engine ()
2016-12-20 09:03:27,501 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2016-12-20 09:03:27,502 INFO sqlalchemy.engine.base.Engine ()
2016-12-20 09:03:27,503 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("winners")
2016-12-20 09:03:27,505 INFO sqlalchemy.engine.base.Engine ()
2016-12-20 09:03:27,507 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE winners (
	id INTEGER NOT NULL, 
	name VARCHAR, 
	category VARCHAR, 
	year INTEGER, 
	nationality VARCHAR, 
	sex VARCHAR(6), 
	PRIMARY KEY (id), 
	CHECK (sex IN ('male', 'female'))
)


2016-12-20 09:03:27,508 INFO sqlalchemy.engine.base.Engine ()
2016-12-20 09:03:27,511 INFO sqlalchemy.engine.base.Engine COMMIT


In [129]:
import sqlalchemy.orm as sorm

In [130]:
Session = sorm.sessionmaker(bind=engine)
session = Session()

In [131]:
data = [{'category': 'Physics',
         'name': 'Albert Einstein',
         'nationality': 'Swiss',
         'sex': 'male',
         'year': 1921},
        {'category': 'Physics',
         'name': 'Paul Dirac',
         'nationality': 'British',
         'sex': 'male',
         'year': 1933},
        {'category': 'Chemistry',
         'name': 'Marie Curie',
         'nationality': 'Polish',
         'sex': 'female',
         'year': 1911}]
data

[{'category': 'Physics',
  'name': 'Albert Einstein',
  'nationality': 'Swiss',
  'sex': 'male',
  'year': 1921},
 {'category': 'Physics',
  'name': 'Paul Dirac',
  'nationality': 'British',
  'sex': 'male',
  'year': 1933},
 {'category': 'Chemistry',
  'name': 'Marie Curie',
  'nationality': 'Polish',
  'sex': 'female',
  'year': 1911}]

In [132]:
albert = Winner(**data[0])
session.add(albert)
session.new

IdentitySet([<Winner(name='Albert Einstein', category='Physics', year='1921')])

In [133]:
session.expunge(albert)
session.new

IdentitySet([])

In [134]:
winner_rows = [Winner(**w) for w in data]
session.add_all(winner_rows)
session.commit()

2016-12-20 09:03:27,565 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2016-12-20 09:03:27,568 INFO sqlalchemy.engine.base.Engine INSERT INTO winners (name, category, year, nationality, sex) VALUES (?, ?, ?, ?, ?)
2016-12-20 09:03:27,570 INFO sqlalchemy.engine.base.Engine ('Albert Einstein', 'Physics', 1921, 'Swiss', 'male')
2016-12-20 09:03:27,571 INFO sqlalchemy.engine.base.Engine INSERT INTO winners (name, category, year, nationality, sex) VALUES (?, ?, ?, ?, ?)
2016-12-20 09:03:27,572 INFO sqlalchemy.engine.base.Engine ('Paul Dirac', 'Physics', 1933, 'British', 'male')
2016-12-20 09:03:27,573 INFO sqlalchemy.engine.base.Engine INSERT INTO winners (name, category, year, nationality, sex) VALUES (?, ?, ?, ?, ?)
2016-12-20 09:03:27,574 INFO sqlalchemy.engine.base.Engine ('Marie Curie', 'Chemistry', 1911, 'Polish', 'female')
2016-12-20 09:03:27,575 INFO sqlalchemy.engine.base.Engine COMMIT


In [135]:
session.query(Winner).count()

2016-12-20 09:03:27,588 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2016-12-20 09:03:27,590 INFO sqlalchemy.engine.base.Engine SELECT count(*) AS count_1 
FROM (SELECT winners.id AS winners_id, winners.name AS winners_name, winners.category AS winners_category, winners.year AS winners_year, winners.nationality AS winners_nationality, winners.sex AS winners_sex 
FROM winners) AS anon_1
2016-12-20 09:03:27,591 INFO sqlalchemy.engine.base.Engine ()


3

In [136]:
result = session.query(Winner).filter_by(nationality = 'Swiss')

In [137]:
list(result)

2016-12-20 09:03:27,605 INFO sqlalchemy.engine.base.Engine SELECT winners.id AS winners_id, winners.name AS winners_name, winners.category AS winners_category, winners.year AS winners_year, winners.nationality AS winners_nationality, winners.sex AS winners_sex 
FROM winners 
WHERE winners.nationality = ?
2016-12-20 09:03:27,606 INFO sqlalchemy.engine.base.Engine ('Swiss',)


[<Winner(name='Albert Einstein', category='Physics', year='1921')]

In [138]:
list(session.query(Winner).filter(Winner.nationality == 'Swiss'))

2016-12-20 09:03:27,617 INFO sqlalchemy.engine.base.Engine SELECT winners.id AS winners_id, winners.name AS winners_name, winners.category AS winners_category, winners.year AS winners_year, winners.nationality AS winners_nationality, winners.sex AS winners_sex 
FROM winners 
WHERE winners.nationality = ?
2016-12-20 09:03:27,618 INFO sqlalchemy.engine.base.Engine ('Swiss',)


[<Winner(name='Albert Einstein', category='Physics', year='1921')]

In [139]:
result = session.query(Winner).filter(Winner.category == 'Physics',
                                      Winner.category != 'Swiss')
list(result)

2016-12-20 09:03:27,626 INFO sqlalchemy.engine.base.Engine SELECT winners.id AS winners_id, winners.name AS winners_name, winners.category AS winners_category, winners.year AS winners_year, winners.nationality AS winners_nationality, winners.sex AS winners_sex 
FROM winners 
WHERE winners.category = ? AND winners.category != ?
2016-12-20 09:03:27,627 INFO sqlalchemy.engine.base.Engine ('Physics', 'Swiss')


[<Winner(name='Albert Einstein', category='Physics', year='1921'),
 <Winner(name='Paul Dirac', category='Physics', year='1933')]

In [140]:
session.query(Winner).get(3)

2016-12-20 09:03:27,635 INFO sqlalchemy.engine.base.Engine SELECT winners.id AS winners_id, winners.name AS winners_name, winners.category AS winners_category, winners.year AS winners_year, winners.nationality AS winners_nationality, winners.sex AS winners_sex 
FROM winners 
WHERE winners.id = ?
2016-12-20 09:03:27,636 INFO sqlalchemy.engine.base.Engine (3,)


<Winner(name='Marie Curie', category='Chemistry', year='1911')

In [141]:
result = session.query(Winner).order_by('year')
list(result)

2016-12-20 09:03:27,645 INFO sqlalchemy.engine.base.Engine SELECT winners.id AS winners_id, winners.name AS winners_name, winners.category AS winners_category, winners.year AS winners_year, winners.nationality AS winners_nationality, winners.sex AS winners_sex 
FROM winners ORDER BY winners.year
2016-12-20 09:03:27,647 INFO sqlalchemy.engine.base.Engine ()


[<Winner(name='Marie Curie', category='Chemistry', year='1911'),
 <Winner(name='Albert Einstein', category='Physics', year='1921'),
 <Winner(name='Paul Dirac', category='Physics', year='1933')]

In [142]:
def sa_inst_to_dict(inst, remove_id=True):
    d = {}
    
    for column in inst.__table__.columns:
        d[column.name] = getattr(inst, column.name)
    
    if remove_id:
        d.pop('id')

    return d

In [143]:
winner_rows = session.query(Winner)
nobel_winners = [sa_inst_to_dict(w) for w in winner_rows]
nobel_winners

2016-12-20 09:03:27,665 INFO sqlalchemy.engine.base.Engine SELECT winners.id AS winners_id, winners.name AS winners_name, winners.category AS winners_category, winners.year AS winners_year, winners.nationality AS winners_nationality, winners.sex AS winners_sex 
FROM winners
2016-12-20 09:03:27,666 INFO sqlalchemy.engine.base.Engine ()


[{'category': 'Physics',
  'name': 'Albert Einstein',
  'nationality': 'Swiss',
  'sex': 'male',
  'year': 1921},
 {'category': 'Physics',
  'name': 'Paul Dirac',
  'nationality': 'British',
  'sex': 'male',
  'year': 1933},
 {'category': 'Chemistry',
  'name': 'Marie Curie',
  'nationality': 'Polish',
  'sex': 'female',
  'year': 1911}]

In [144]:
marie = session.query(Winner).get(3)
marie.nationality = 'French'
session.dirty

IdentitySet([<Winner(name='Marie Curie', category='Chemistry', year='1911')])

In [145]:
session.commit()

2016-12-20 09:03:27,684 INFO sqlalchemy.engine.base.Engine UPDATE winners SET nationality=? WHERE winners.id = ?
2016-12-20 09:03:27,686 INFO sqlalchemy.engine.base.Engine ('French', 3)
2016-12-20 09:03:27,688 INFO sqlalchemy.engine.base.Engine COMMIT


In [146]:
session.dirty

IdentitySet([])

In [147]:
session.query(Winner).get(3).nationality

2016-12-20 09:03:27,703 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2016-12-20 09:03:27,705 INFO sqlalchemy.engine.base.Engine SELECT winners.id AS winners_id, winners.name AS winners_name, winners.category AS winners_category, winners.year AS winners_year, winners.nationality AS winners_nationality, winners.sex AS winners_sex 
FROM winners 
WHERE winners.id = ?
2016-12-20 09:03:27,707 INFO sqlalchemy.engine.base.Engine (3,)


'French'

In [148]:
#session.query(Winner).filter_by(name='Albert Einstein').delete()

In [149]:
list(session.query(Winner))

2016-12-20 09:03:27,723 INFO sqlalchemy.engine.base.Engine SELECT winners.id AS winners_id, winners.name AS winners_name, winners.category AS winners_category, winners.year AS winners_year, winners.nationality AS winners_nationality, winners.sex AS winners_sex 
FROM winners
2016-12-20 09:03:27,725 INFO sqlalchemy.engine.base.Engine ()


[<Winner(name='Albert Einstein', category='Physics', year='1921'),
 <Winner(name='Paul Dirac', category='Physics', year='1933'),
 <Winner(name='Marie Curie', category='Chemistry', year='1911')]

In [150]:
#Winner.__table__.drop(engine)